데이터 전처리 과정 분석

convert_h36m.py 코드를 통해 h36m_sh_conf_cam_source_final.pkl 파일로부터 clip들을 나누어 data/motion3d/MB3D_f243s81/H36M-SH 폴더에 저장함
그 과정에서 전처리 괒어을 거쳐 원본 파일과 새로 생성된 파일의 값이 달라짐 -> 이 부분을 이해해야 AI Hub 데이터를 동일한 형식으로 만들어 훈련시킬 수 있음

In [18]:
import pickle
import copy
import numpy as np

# 원본 파일
with open('h36m_sh_conf_cam_source_final.pkl', 'rb') as f:
	datareader = pickle.load(f)
 
# 예제 CLIP 파일
with open('./MB3D_f243s81/H36M-SH/test/00000000.pkl', 'rb') as f:
	test = pickle.load(f)

### test/00000000.pkl 분석

In [19]:
test.keys()

dict_keys(['data_input', 'data_label'])

In [20]:
test['data_input'].shape # 2d + confidence (243, 17, 3)
test['data_label'].shape # 3d (243, 17, 3)

(243, 17, 3)

### h36m_sh_conf_cam_source_final.pkl 분석

In [21]:
datareader.keys()

dict_keys(['train', 'test'])

#### train

In [23]:
datareader['train'].keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'camera_name', 'source'])

In [24]:
datareader['train']['joint_2d'].shape # (1559752, 17, 2) array
datareader['train']['confidence'].shape # (1559752, 17, 1) array
datareader['train']['joint3d_image'].shape # (1559752, 17, 3)
datareader['train']['camera_name'].shape # (1559752,) array
len(datareader['train']['source']) # 1559752 list

1559752

In [25]:
datareader['train']['joint_2d'][0]

array([[472., 444.],
       [500., 447.],
       [479., 530.],
       [507., 622.],
       [448., 444.],
       [459., 536.],
       [469., 633.],
       [490., 399.],
       [483., 341.],
       [479., 316.],
       [486., 296.],
       [455., 358.],
       [431., 416.],
       [414., 457.],
       [510., 354.],
       [521., 413.],
       [517., 461.]], dtype=float32)

#### test

In [6]:
datareader['test'].keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'joints_2.5d_image', '2.5d_factor', 'camera_name', 'action', 'source'])

In [8]:
datareader['test']['joint_2d'].shape # (566920, 17, 2) array
datareader['test']['confidence'].shape # (566920, 17, 1) array
datareader['test']['joint3d_image'].shape # (566920, 17, 3) array
datareader['test']['joints_2.5d_image'].shape # (566920, 17, 3) array = 2.5d_factor * joint3d_image
datareader['test']['2.5d_factor'].shape # (566920,) array
datareader['test']['camera_name'].shape # (566920,) array
len(datareader['test']['action']) # 566920 list
len(datareader['test']['source']) # 566920 list

566920

#### joint_2.5d_image, 2.5d_factor, joint3d_image 관계
joint3d_image * 2.5d_factor = joint_2.5d_image

In [17]:
datareader['test']['camera_name'][0], datareader['test']['action'][0], datareader['test']['source'][0]

('54138969', 'Direction', 's_09_act_02_subact_01_ca_01')

In [9]:
print(datareader['test']['joints_2.5d_image'][0][0], "\n\n",
      datareader['test']['2.5d_factor'][0], "\n\n",
      datareader['test']['joint3d_image'][0][0], "\n\n",
      datareader['test']['2.5d_factor'][0] * datareader['test']['joint3d_image'][0][0])

[2099.9255 1888.2003    0.    ] 

 4.656527 

 [450.9639 405.4954   0.    ] 

 [2099.9255 1888.2003    0.    ]


In [10]:
datareader['test']['joint_2d'][0] 

array([[453., 404.],
       [473., 404.],
       [483., 499.],
       [489., 594.],
       [427., 408.],
       [420., 506.],
       [414., 601.],
       [456., 352.],
       [453., 293.],
       [453., 280.],
       [456., 254.],
       [417., 300.],
       [358., 316.],
       [302., 319.],
       [486., 300.],
       [545., 316.],
       [584., 322.]], dtype=float32)

In [16]:
# joints_2.5d_image x, y 
datareader['test']['joints_2.5d_image'][0][:, :2] / datareader['test']['2.5d_factor'][0] - datareader['test']['joint_2d'][0] 

array([[-2.0361023 ,  1.4953918 ],
       [ 2.4246826 , -1.4642639 ],
       [-3.1942444 ,  0.89019775],
       [ 2.0151978 ,  1.512146  ],
       [-0.8762207 ,  0.500885  ],
       [ 1.4437561 ,  1.4277649 ],
       [ 4.976654  ,  4.6240234 ],
       [-3.125824  , -0.4510193 ],
       [ 0.7156677 ,  4.722809  ],
       [-1.3098145 , -6.461609  ],
       [ 0.46759033, -2.524826  ],
       [ 1.6781311 ,  1.2906494 ],
       [-0.14968872, -0.5526123 ],
       [-0.06716919, -0.95196533],
       [ 4.5478516 ,  2.8391418 ],
       [ 0.63775635,  1.5794373 ],
       [-0.03808594,  0.60461426]], dtype=float32)

### 2D Input Data Preprocessing

In [11]:
# convert_h36m.py에 의해 00000000.pkl로 추출된 2d 입력 데이터 
test['data_input'][0], test['data_input'].shape, test['data_label'].shape

(array([[-0.09399998, -0.19399998,  0.65554845],
        [-0.05400002, -0.19399998,  0.82664   ],
        [-0.03399998, -0.00399997,  0.77685416],
        [-0.02200001,  0.18599996,  0.91304016],
        [-0.14600003, -0.18600002,  0.8345239 ],
        [-0.16000003,  0.00999996,  0.8205779 ],
        [-0.17199999,  0.20000002,  0.7974426 ],
        [-0.088     , -0.298     ,  0.723265  ],
        [-0.09399998, -0.41599998,  0.720955  ],
        [-0.09399998, -0.442     ,  0.8918598 ],
        [-0.088     , -0.494     ,  0.9627786 ],
        [-0.16600001, -0.40199998,  0.66214174],
        [-0.28399998, -0.36999997,  0.8186033 ],
        [-0.39600003, -0.364     ,  0.8860159 ],
        [-0.028     , -0.40199998,  0.78756124],
        [ 0.09000003, -0.36999997,  0.7411108 ],
        [ 0.16799998, -0.358     ,  0.8789877 ]], dtype=float32),
 (243, 17, 3),
 (243, 17, 3))

In [12]:
# h36m_sh_conf_cam_source_final.pkl에서 2d 데이터를 전처리하는 과정 (normalize)
camera_name = datareader['test']['camera_name'][0]
if camera_name == '54138969' or camera_name == '60457274':
    res_w, res_h = 1000, 1002
elif camera_name == '55011271' or camera_name == '58860488':
    res_w, res_h = 1000, 1000
print(res_w, res_h)

# preprocessing
datareader['test']['joint_2d'][0]  / res_w * 2 - [1, res_h / res_w]

1000 1002


array([[-0.09399998, -0.19399997],
       [-0.05400002, -0.19399997],
       [-0.03399998, -0.00399997],
       [-0.02200001,  0.18599996],
       [-0.14600003, -0.18600002],
       [-0.16000003,  0.00999996],
       [-0.17199999,  0.20000002],
       [-0.088     , -0.298     ],
       [-0.09399998, -0.41599997],
       [-0.09399998, -0.442     ],
       [-0.088     , -0.49399998],
       [-0.16600001, -0.40199998],
       [-0.28399998, -0.36999997],
       [-0.39600003, -0.36399999],
       [-0.028     , -0.40199998],
       [ 0.09000003, -0.36999997],
       [ 0.16799998, -0.35800001]])

In [13]:
datareader['test']['joint_2d'][0]

array([[453., 404.],
       [473., 404.],
       [483., 499.],
       [489., 594.],
       [427., 408.],
       [420., 506.],
       [414., 601.],
       [456., 352.],
       [453., 293.],
       [453., 280.],
       [456., 254.],
       [417., 300.],
       [358., 316.],
       [302., 319.],
       [486., 300.],
       [545., 316.],
       [584., 322.]], dtype=float32)

두 결과가 동일함을 확인할 수 있다

### 3D Output Data Preprocessing

In [12]:
# 추출된 3d gt
test['data_label'][0]

array([[-0.09807223, -0.1910092 ,  0.        ],
       [-0.04915065, -0.19692853,  0.01765747],
       [-0.04038846, -0.00221958,  0.04414008],
       [-0.01796961,  0.1890243 ,  0.04464229],
       [-0.14775246, -0.18499824, -0.01765808],
       [-0.15711248,  0.0128555 ,  0.00503382],
       [-0.16204667,  0.20924804, -0.00685673],
       [-0.09425163, -0.29890212, -0.02466628],
       [-0.09256876, -0.40655443, -0.05074621],
       [-0.09661967, -0.45492324, -0.04101023],
       [-0.0870648 , -0.49904963, -0.07057631],
       [-0.16264373, -0.39941868, -0.08863481],
       [-0.28429937, -0.37110522, -0.13213187],
       [-0.39613432, -0.36590394, -0.14020935],
       [-0.01890427, -0.3963218 , -0.02291151],
       [ 0.09127533, -0.3668411 ,  0.03014895],
       [ 0.16792381, -0.3567908 ,  0.09531134]], dtype=float32)

In [13]:
# 원본 3d gt를 전처리 (normalize)
test_label = copy.deepcopy(datareader['test']['joint3d_image'][0])
# preprocessing
test_label[:, :2] = test_label[:, :2] / res_w * 2 - [1, res_h / res_w]
test_label[:, 2:] = test_label[:, 2:] / res_w * 2
test_label

array([[-0.09807223, -0.1910092 ,  0.        ],
       [-0.04915065, -0.19692853,  0.01765747],
       [-0.04038846, -0.00221958,  0.04414008],
       [-0.01796961,  0.1890243 ,  0.04464229],
       [-0.14775246, -0.18499824, -0.01765808],
       [-0.15711248,  0.0128555 ,  0.00503382],
       [-0.16204667,  0.20924804, -0.00685673],
       [-0.09425163, -0.29890212, -0.02466628],
       [-0.09256876, -0.40655443, -0.05074621],
       [-0.09661967, -0.45492324, -0.04101023],
       [-0.0870648 , -0.49904963, -0.07057631],
       [-0.16264373, -0.39941868, -0.08863481],
       [-0.28429937, -0.37110522, -0.13213187],
       [-0.39613432, -0.36590394, -0.14020935],
       [-0.01890427, -0.3963218 , -0.02291151],
       [ 0.09127533, -0.3668411 ,  0.03014895],
       [ 0.16792381, -0.3567908 ,  0.09531134]], dtype=float32)

In [14]:
# denormalize
d = copy.deepcopy(test['data_label'][0])
print(d)
d[:, :2] = (d[:, :2] + np.array([1, res_h / res_w])) * res_w / 2
d[:,2:] = d[:,2:] * res_w / 2
d

[[-0.09807223 -0.1910092   0.        ]
 [-0.04915065 -0.19692853  0.01765747]
 [-0.04038846 -0.00221958  0.04414008]
 [-0.01796961  0.1890243   0.04464229]
 [-0.14775246 -0.18499824 -0.01765808]
 [-0.15711248  0.0128555   0.00503382]
 [-0.16204667  0.20924804 -0.00685673]
 [-0.09425163 -0.29890212 -0.02466628]
 [-0.09256876 -0.40655443 -0.05074621]
 [-0.09661967 -0.45492324 -0.04101023]
 [-0.0870648  -0.49904963 -0.07057631]
 [-0.16264373 -0.39941868 -0.08863481]
 [-0.28429937 -0.37110522 -0.13213187]
 [-0.39613432 -0.36590394 -0.14020935]
 [-0.01890427 -0.3963218  -0.02291151]
 [ 0.09127533 -0.3668411   0.03014895]
 [ 0.16792381 -0.3567908   0.09531134]]


array([[450.9639   , 405.4954   ,   0.       ],
       [475.42468  , 402.53574  ,   8.828734 ],
       [479.80576  , 499.8902   ,  22.07004  ],
       [491.0152   , 595.51215  ,  22.321146 ],
       [426.12378  , 408.5009   ,  -8.829042 ],
       [421.44376  , 507.42776  ,   2.5169103],
       [418.97665  , 605.624    ,  -3.4283628],
       [452.87418  , 351.54895  , -12.3331375],
       [453.71564  , 297.72278  , -25.373106 ],
       [451.69016  , 273.5384   , -20.505117 ],
       [456.4676   , 251.47519  , -35.288155 ],
       [418.67813  , 301.29065  , -44.317406 ],
       [357.8503   , 315.4474   , -66.06594  ],
       [301.93283  , 318.04803  , -70.104675 ],
       [490.54785  , 302.8391   , -11.455753 ],
       [545.6377   , 317.57944  ,  15.074472 ],
       [583.9619   , 322.60458  ,  47.65567  ]], dtype=float32)